This notebook tests the web scraping code.

In [2]:
import requests
from bs4 import BeautifulSoup
import networkx as nx
from gensim import corpora
import nltk
from nltk.corpus import stopwords
from gensim.models.ldamodel import LdaModel
import re
from urllib.robotparser import RobotFileParser
from urllib.parse import urljoin

In [ ]:
# Make sure to download NLTK stopwords
nltk.download('stopwords')

# Define stopwords
stop_words = set(stopwords.words('english'))

# Initialize graph
graph = nx.DiGraph()

# Helper function to clean text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    words = text.lower().split()
    return [word for word in words if word not in stop_words]